#imports

In [ ]:
!pip install tqdm

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler, StandardScaler,RobustScaler
import pandas as pd

import matplotlib.pyplot as plt

import tqdm
from tqdm import tqdm

import numpy as np

#Config Dic

In [3]:
class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

In [4]:
config = dotdict()

config.input_size = 6 #(month, day,hour , ...)
config.output_size = 3

config.hidden_size = 256
config.num_layers =2 

config.seq_len = 96
config.pred_len = 12

config.random_seed= 1

config.learning_rate = 0.001

config.batch_size = 32
config.epochs = 500



config.early_stopping = 8
config.Model = 'GB_MyModel_'

# Data
### read, preprocess, split, scale, Create Daatasets

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [7]:
def Read_Data():

  data = pd.read_csv('/content/drive/MyDrive/Data_FFT_Model/GB_GBN_withoutTemp_Wind.csv')

  data['date'] = pd.to_datetime(data.date)
  data['year'] = data['date'].dt.year
  data['month'] = data['date'].dt.month
  data['day'] = data['date'].dt.day
  data['hour'] = data['date'].dt.hour

  data = data.drop('date', axis=1)
  data = data.drop('year', axis=1)

  #DROP NULL
  print('data_shape BEFORE drop null' ,data.shape)
  #Remove Missing Values
  data = data.dropna(axis=0, how='any')
  print('data_shape AFTER drop null' ,data.shape)

  return data

In [8]:
def Data_Cleaning(data):

  threshold = 2
  print('first shape =',data.shape)

  #Remove Noise and Outliers
  data =data[(data['Price']>0)] 
  print('After prc remove = ', data.shape)

  return data

In [9]:
def Data_Split_Scales(data, scale):
  #split
  train_size = int( len(data) * 0.8 )
  test_size = len(data) - train_size
  train, test = data[:train_size], data[train_size:]

  #scale
  if scale:
    scaler = StandardScaler() #MinMaxScaler()#RobustScaler#ُ
    train = scaler.fit_transform(train.values)
    test = scaler.fit_transform(test.values)

  return train, test


In [ ]:
data = Read_Data()
data = Data_Cleaning(data)

In [11]:
train_data, test_data = Data_Split_Scales(data, scale =True)

In [12]:
train_data, eval_data = Data_Split_Scales(train_data, scale =False)

In [ ]:
print('train.shape :', train_data.shape)
print('eval_data.shape :', eval_data.shape)
print('test.shape :', test_data.shape)

In [14]:
def MyCreate_dataset(data, seq_len, pred_len, pred_size, step):
  x, y =[], []

  for i in range(0, len(data) - seq_len - pred_len , step):
    feature = data[i: i+seq_len]
    target = data[i+seq_len: i+seq_len+pred_len]

    target = target[:, 0:pred_size]


    x.append(feature)
    y.append(target)

  return torch.tensor(x), torch.tensor(y)

#Encoder_Scaled Attention_Decoder (Seq2Seq Model)

In [99]:
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers=num_layers, bidirectional=True, batch_first=True, dropout=dropout)
        self.fc_hidden = nn.Linear(hidden_size * 2 * num_layers, hidden_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input):
        output_enc, (hidden, cell) = self.lstm(input)  # [batch, seq_len, 2*hidden_size]
        hidden_1dim = self.fc_hidden(hidden.permute(1, 0, 2).reshape(hidden.shape[1], -1))  # [batch, hidden_size]
        output_enc = self.dropout(output_enc)
        return output_enc, hidden, cell, hidden_1dim

In [100]:
class Decoder(nn.Module):
    def __init__(self, hidden_size, output_size, num_layers, dropout):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers

        # Attention mechanism
        self.query = nn.Linear(2 * hidden_size, hidden_size)
        self.key = nn.Linear(2 * hidden_size, hidden_size)
        self.value = nn.Linear(2 * hidden_size, 2 * hidden_size)
        self.dropout = nn.Dropout(dropout)

        # LSTM and output layers
        self.lstm = nn.LSTM(2 * hidden_size + output_size, hidden_size, num_layers=num_layers, bidirectional=True, dropout=dropout)
        self.fc = nn.Linear(2 * hidden_size, output_size)

    def forward(self, input, encoder_state, hidden_decoder, cell_decoder, hidden_1dim):
        # Prepare attention mechanism
        query = self.query(encoder_state)  # [batch, seq_len, hidden_size]
        key = self.key(encoder_state)      # [batch, seq_len, hidden_size]
        value = self.value(encoder_state)  # [batch, seq_len, 2*hidden_size]

        scores = torch.matmul(query, key.transpose(-2, -1)) / (self.hidden_size ** 0.5)  # [batch, seq_len, seq_len]
        attn_weights = nn.functional.softmax(scores, dim=-1)  # [batch, seq_len, seq_len]
        attn_weights = self.dropout(attn_weights)

        context_vector = torch.bmm(attn_weights, value)  # [batch, seq_len, 2*hidden_size]
        context_vector = context_vector[:, -1, :].unsqueeze(1)  # [batch, 1, 2*hidden_size]

        # Concatenate context vector with input
        dec_input = torch.cat((context_vector, input.unsqueeze(1)), dim=2)  # [batch, 1, 2*hidden_size + output_size]

        # LSTM step
        output_dec, (hidden_decoder, cell_decoder) = self.lstm(dec_input.permute(1, 0, 2), (hidden_decoder, cell_decoder))
        prediction = self.fc(output_dec.squeeze(0))  # [batch, output_size]

        return prediction, hidden_decoder, cell_decoder

In [101]:
class Seq2Seq(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, device):
        super(Seq2Seq, self).__init__()
        self.encoder = Encoder(input_size, hidden_size, num_layers, dropout=0.1).to(device)
        self.decoder = Decoder(hidden_size, output_size, num_layers, dropout=0.1).to(device)
        self.device = device

    def forward(self, source, target=None):
        batch_size = source.shape[0]
        target_len = target.shape[1] if target is not None else config['pred_len']
        outputs = torch.zeros(target_len, batch_size, config['output_size']).to(self.device)

        # Encoder pass
        out_encoder, hidden, cell, hidden_1dim = self.encoder(source.float())

        # Initialize decoder input
        dec_input = torch.zeros(batch_size, config['output_size']).to(self.device)  # Start token or zeros

        for t in range(target_len):
            teacher_forcing = torch.rand(1) < 0.5 if target is not None else False

            if teacher_forcing and target is not None:
                dec_input = target[:, t, :]  # Use ground truth as next input
            else:
                dec_input = outputs[t - 1] if t > 0 else dec_input  # Use previous prediction

            # Decoder pass
            prediction, hidden, cell = self.decoder(
                dec_input, out_encoder, hidden, cell, hidden_1dim.unsqueeze(0)
            )
            outputs[t] = prediction

        return outputs.permute(1, 0, 2)  # [batch, pred_len, output_size]

#Create Model

In [102]:
momentum = 0 #0.9
alpha = 0.99
eps = 1e-08

model = Seq2Seq(config['input_size'], config['hidden_size'], config['output_size'],
                config['num_layers'], device).to(device)

criteria = nn.MSELoss()

optimizer = torch.optim.Adam(model.parameters(), lr = config['learning_rate'])



#train phases

In [ ]:
torch.cuda.memory_allocated(device=device)

In [104]:
import random

def set_seed(seed):
    np.random.seed(seed)          # for numpy
    random.seed(seed)             # for random 
    torch.manual_seed(seed)       # for PyTorch (CPU)
    torch.cuda.manual_seed(seed)  # for PyTorch (GPU)
    torch.cuda.manual_seed_all(seed)  #for Multi GPU
    torch.backends.cudnn.deterministic = True  # CuDNN
    torch.backends.cudnn.benchmark = False     # CuDNN

In [105]:
Random_seed = config['random_seed']
set_seed(Random_seed)

In [ ]:
#Train ______________
x_train, y_train = MyCreate_dataset(train_data, config['seq_len'], config['pred_len'], config['output_size'], step=1)
print('x_train :' , x_train.shape)
print('y_train :' , y_train.shape)

x_train_fft = torch.fft.fft(torch.tensor(x_train))

x_train_fft = torch.real(x_train_fft)

train_dataloader = torch.utils.data.DataLoader(
    torch.utils.data.TensorDataset(x_train_fft, y_train),
    shuffle=True,
    batch_size=config['batch_size'],
    drop_last=True,
    num_workers=0,
    worker_init_fn=lambda _: np.random.seed(Random_seed)
)



In [ ]:
#Eval ______________
x_eval, y_eval = MyCreate_dataset(eval_data, config['seq_len'], config['pred_len'], config['output_size'], step=1)
print('x_eval :' , x_eval.shape)
print('y_eval :' , y_eval.shape)

x_eval_fft = torch.fft.fft(torch.tensor(x_eval))

x_eval_fft = torch.real(x_eval_fft)


eval_dataloader = torch.utils.data.DataLoader(
    torch.utils.data.TensorDataset(x_eval_fft, y_eval),
    shuffle=True,
    batch_size=config['batch_size'],
    drop_last=True,
    num_workers=0,
    worker_init_fn=lambda _: np.random.seed(Random_seed)
)


#Train

In [108]:
def train(model, train_loader, val_loader, criterion, optimizer, device):
    model.train()
    train_loss = 0
    val_losses = []
    train_losses = []
    best_val_loss = float('inf')

    stop_cheack = 0

    for epoch in range(config['epochs']):
        print('\n********************\n')
        train_loss = 0
        for batch_idx, (data, target) in enumerate(tqdm(train_loader, desc='Train')):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data.float(), target.float())
            output = output.to(device)

            loss = criterion(output.float(), target.float())

            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            train_loss_2 = train_loss / len(train_loader)


        # Evaluate on validation set
        val_loss = 0
        with torch.no_grad():
            for batch_idx, (data, target) in enumerate(tqdm(val_loader, desc='Validating')):
                data, target = data.to(device), target.to(device)
                output = model(data.float(), target.float())
                output = output.to(device)
                val_loss += criterion(output.float(), target.float()).item()
        val_loss /= len(val_loader)
        val_losses.append(val_loss)


        # Check if the validation loss has improved
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            filename = f"{config['Model']}_seed_pred {config['pred_len']}_{config['random_seed']}_ep {epoch}_seq {config['seq_len']}.pth"
            torch.save(model.state_dict(), '/content/drive/MyDrive/Data_FFT_Model/MyModel_4Esfand1403/'+ filename)
            stop_cheack = 0
        else:
          stop_cheack += 1

        print('==========================')
        print(f"Epoch {epoch}/{config['epochs']}:  \n Train Loss: {train_loss:.4f}, tain Loss scale = {train_loss_2:.4},  Val Loss: {val_loss:.4f}")
        print('_____________\n', '\nAvg _train_Loss =', train_loss / len(train_loader), '\nAvg_val_loss =', val_loss)
        los_ = train_loss / len(train_loader)
        train_losses.append(los_)

        # Check early stopping criteria
        if stop_cheack >= config['early_stopping']:
            print("****************** Early stopping *************************")
            print(f"Epoch {epoch+1}/{config['epochs']}")
            break

    # Plot train and validation losses
    plt.plot(train_losses, label='Train loss')
    plt.plot(val_losses, label='Validation loss')
    plt.legend()
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.show()

    return train_loss / len(train_loader), train_losses, val_losses

In [ ]:

avg_loss, loss_array , val_losses= train(model, train_dataloader, eval_dataloader, criteria, optimizer, device)
print('------------------- train is complete -------------------------')
print('avg_loss =' , avg_loss)

In [ ]:
losses_cpu = torch.tensor(loss_array, device='cuda').cpu()#.tolist()
val_losses_cpu = torch.tensor(val_losses, device='cuda').cpu()#.tolist()


# رسم نمودار تغییرات لاس در هر مرحله
plt.plot(losses_cpu)
plt.plot(val_losses_cpu)
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.show()